In [9]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
from ultralytics import YOLO
import cv2
import tensorflow as tf
import xml.etree.ElementTree as ET
import time

In [2]:
FILE_ROOT = "J:\\jupyter\\plate_recognition\\car_plate_fa"
IMAGE_PATH = FILE_ROOT + "\\images"  
ANNOTATIONS_PATH = FILE_ROOT + "\\annotations"

DATA_ROOT = "datasets\\Dataset_fa\\"
DEST_IMAGES_PATH = "train\\images"
DEST_LABELS_PATH = "train\\labels"


In [20]:
from PIL import Image

def get_image_dimensions(file_path):
    with Image.open(file_path) as img:
        width, height = img.size
    return width, height

In [60]:
# base="J:/jupyter/plate_recognition/car_plate_fa/annotations"
# for file in os.listdir(base):
#     img_path="J:/jupyter/plate_recognition/car_plate_fa/images/" + file.replace("xml","jpg")
#     w,h=get_image_dimensions(img_path)
#     tree = ET.parse(base+"/"+file)
#     root = tree.getroot()
#     size = ET.Element('size')
#     width = ET.Element('width')
#     width.text =str(w)
#     height = ET.Element('height')
#     height.text = str(h)
#     depth = ET.Element('depth')
#     depth.text = '3'
    
#     # Add the elements to <size>
#     size.append(width)
#     size.append(height)
#     size.append(depth)
    
#     # Find <annotation> and append <size> as its child
#     # annotation = root.find('annotation')
#     # annotation.append(size)
#     root.append(size)
    
#     # Write the modified XML back to the file
#     tree.write(base+"/"+file)

In [3]:
def resize_with_aspect_ratio(image, new_height):
    # Calculate the aspect ratio
    aspect_ratio = image.shape[1] / image.shape[0]
    
    # Calculate the new width based on the desired height and aspect ratio
    new_width = int(new_height * aspect_ratio)
    
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height))
    
    return resized_image
    
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1,y1,x2,y2=box.astype(np.int32)
    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]
    
def save_file(img_jpg_file_name, size, img_box):
    classes = ['license']
    save_file_name = DATA_ROOT + DEST_LABELS_PATH + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:                  
        #cls_num = classes.index(box[0]) # find class_id
        cls_num = 0
        new_box = cord_converter(size, np.array(box[1:]) ) # convert box coord into YOLO x,y,w,h

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()

def get_bbox_from_xml(box):
    corners=["xmin","ymin","xmax","ymax"]
    corners_numbers=[]
    for c in corners:
        data=int(float(box.getElementsByTagName(c)[0].childNodes[0].data))
        corners_numbers.append(data)
    return corners_numbers
        
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data

    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        if cls_name=="کل ناحیه پلاک":
            x1,y1,x2,y2=get_bbox_from_xml(box)
            print("box:(xmin,ymin,xmax,ymax)", x1, y1, x2, y2)
            img_box.append(["license", x1, y1, x2, y2])
        
    save_file(img_xml_file, [img_w, img_h], img_box)

In [27]:
  # img_name = file.split(".")[0]
    # xml_path = ANNOTATIONS_PATH + '/' + file 
    # img_path=IMAGE_PATH +"/" + img_name +".jpg"
    # w,h=get_image_dimensions(img_path)
    # tree = ET.parse(xml_path)
    # print(tree)
    # root = tree.getroot()
    # size = ET.Element('size')
    # width = ET.Element('width')
    # width.text = str(w)
    # height = ET.Element('height')
    # height.text = str(h)
    # depth = ET.Element('depth')
    # depth.text = '3'

    # size.append(width)
    # size.append(height)
    # size.append(depth)
    # root.append(size)
    # tree.write(xml_path)

In [31]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])
    

file name:  day_00004.xml
J:\jupyter\plate_recognition\car_plate_fa\annotations/day_00004.xml
box:(xmin,ymin,xmax,ymax) 496 345 573 364
datasets\Dataset_fa\train\labels/day_00004.txt
file name:  day_00011.xml
J:\jupyter\plate_recognition\car_plate_fa\annotations/day_00011.xml
box:(xmin,ymin,xmax,ymax) 404 369 485 389
datasets\Dataset_fa\train\labels/day_00011.txt
file name:  day_00018.xml
J:\jupyter\plate_recognition\car_plate_fa\annotations/day_00018.xml
box:(xmin,ymin,xmax,ymax) 338 455 592 505
datasets\Dataset_fa\train\labels/day_00018.txt
file name:  day_00024.xml
J:\jupyter\plate_recognition\car_plate_fa\annotations/day_00024.xml
box:(xmin,ymin,xmax,ymax) 278 634 366 661
datasets\Dataset_fa\train\labels/day_00024.txt
file name:  day_00033.xml
J:\jupyter\plate_recognition\car_plate_fa\annotations/day_00033.xml
box:(xmin,ymin,xmax,ymax) 717 345 890 381
datasets\Dataset_fa\train\labels/day_00033.txt
file name:  day_00036.xml
J:\jupyter\plate_recognition\car_plate_fa\annotations/day_0

## YOLOv5 Training

In [5]:
load_path="J:\\jupyter\\plate_recognition\\runs\\detect\\train8\\weights\\best.pt"#CHANGE THIS EVERYTIME


In [6]:

# model=YOLO("yolov8n.yaml",task="detect")
model=YOLO(load_path,task="detect")

In [13]:
results=model.train(data="alpr.yaml", epochs=10, imgsz=640)

Ultralytics YOLOv8.0.188  Python-3.9.17 torch-2.0.1+cpu CPU (Intel Core(TM) i7-4790K 4.00GHz)
engine\trainer: task=detect, mode=train, model=J:\jupyter\plate_recognition\runs\detect\train7\weights\best.pt, data=alpr.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=Tr

In [24]:

img=cv2.imread("J:/jupyter/plate_recognition/car_plate_fa/test/3.jpg")
# img=cv2.imread("J:/jupyter/plate_recognition/carPlate/images/Cars66.png")
# img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
cv2.imshow("win",img)
cv2.waitKey()
cv2.destroyAllWindows()
preds=model.predict(img,save=True)


0: 416x640 1 license, 110.3ms
Speed: 14.8ms preprocess, 110.3ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs\detect\predict22


In [18]:
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
  lw = max(round(sum(image.shape) / 2 * 0.003), 2)
  p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
  cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
  if label:
    tf = max(lw - 1, 1)  # font thickness
    w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
    outside = p1[1] - h >= 3
    p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
    cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(image,
                label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                0,
                lw / 3,
                txt_color,
                thickness=tf,
                lineType=cv2.LINE_AA)
def plot_bboxes(image, boxes, labels=[], colors=[], score=True, conf=None):
  #Define COCO Labels
  if labels == []:
    labels = {0: u'__background__', 1: u'person', 2: u'bicycle',3: u'car', 4: u'motorcycle', 5: u'airplane', 6: u'bus', 7: u'train', 8: u'truck', 9: u'boat', 10: u'traffic light', 11: u'fire hydrant', 12: u'stop sign', 13: u'parking meter', 14: u'bench', 15: u'bird', 16: u'cat', 17: u'dog', 18: u'horse', 19: u'sheep', 20: u'cow', 21: u'elephant', 22: u'bear', 23: u'zebra', 24: u'giraffe', 25: u'backpack', 26: u'umbrella', 27: u'handbag', 28: u'tie', 29: u'suitcase', 30: u'frisbee', 31: u'skis', 32: u'snowboard', 33: u'sports ball', 34: u'kite', 35: u'baseball bat', 36: u'baseball glove', 37: u'skateboard', 38: u'surfboard', 39: u'tennis racket', 40: u'bottle', 41: u'wine glass', 42: u'cup', 43: u'fork', 44: u'knife', 45: u'spoon', 46: u'bowl', 47: u'banana', 48: u'apple', 49: u'sandwich', 50: u'orange', 51: u'broccoli', 52: u'carrot', 53: u'hot dog', 54: u'pizza', 55: u'donut', 56: u'cake', 57: u'chair', 58: u'couch', 59: u'potted plant', 60: u'bed', 61: u'dining table', 62: u'toilet', 63: u'tv', 64: u'laptop', 65: u'mouse', 66: u'remote', 67: u'keyboard', 68: u'cell phone', 69: u'microwave', 70: u'oven', 71: u'toaster', 72: u'sink', 73: u'refrigerator', 74: u'book', 75: u'clock', 76: u'vase', 77: u'scissors', 78: u'teddy bear', 79: u'hair drier', 80: u'toothbrush'}
  #Define colors
  if colors == []:
    #colors = [(6, 112, 83), (253, 246, 160), (40, 132, 70), (205, 97, 162), (149, 196, 30), (106, 19, 161), (127, 175, 225), (115, 133, 176), (83, 156, 8), (182, 29, 77), (180, 11, 251), (31, 12, 123), (23, 6, 115), (167, 34, 31), (176, 216, 69), (110, 229, 222), (72, 183, 159), (90, 168, 209), (195, 4, 209), (135, 236, 21), (62, 209, 199), (87, 1, 70), (75, 40, 168), (121, 90, 126), (11, 86, 86), (40, 218, 53), (234, 76, 20), (129, 174, 192), (13, 18, 254), (45, 183, 149), (77, 234, 120), (182, 83, 207), (172, 138, 252), (201, 7, 159), (147, 240, 17), (134, 19, 233), (202, 61, 206), (177, 253, 26), (10, 139, 17), (130, 148, 106), (174, 197, 128), (106, 59, 168), (124, 180, 83), (78, 169, 4), (26, 79, 176), (185, 149, 150), (165, 253, 206), (220, 87, 0), (72, 22, 226), (64, 174, 4), (245, 131, 96), (35, 217, 142), (89, 86, 32), (80, 56, 196), (222, 136, 159), (145, 6, 219), (143, 132, 162), (175, 97, 221), (72, 3, 79), (196, 184, 237), (18, 210, 116), (8, 185, 81), (99, 181, 254), (9, 127, 123), (140, 94, 215), (39, 229, 121), (230, 51, 96), (84, 225, 33), (218, 202, 139), (129, 223, 182), (167, 46, 157), (15, 252, 5), (128, 103, 203), (197, 223, 199), (19, 238, 181), (64, 142, 167), (12, 203, 242), (69, 21, 41), (177, 184, 2), (35, 97, 56), (241, 22, 161)]
    colors = [(89, 161, 197),(67, 161, 255),(19, 222, 24),(186, 55, 2),(167, 146, 11),(190, 76, 98),(130, 172, 179),(115, 209, 128),(204, 79, 135),(136, 126, 185),(209, 213, 45),(44, 52, 10),(101, 158, 121),(179, 124, 12),(25, 33, 189),(45, 115, 11),(73, 197, 184),(62, 225, 221),(32, 46, 52),(20, 165, 16),(54, 15, 57),(12, 150, 9),(10, 46, 99),(94, 89, 46),(48, 37, 106),(42, 10, 96),(7, 164, 128),(98, 213, 120),(40, 5, 219),(54, 25, 150),(251, 74, 172),(0, 236, 196),(21, 104, 190),(226, 74, 232),(120, 67, 25),(191, 106, 197),(8, 15, 134),(21, 2, 1),(142, 63, 109),(133, 148, 146),(187, 77, 253),(155, 22, 122),(218, 130, 77),(164, 102, 79),(43, 152, 125),(185, 124, 151),(95, 159, 238),(128, 89, 85),(228, 6, 60),(6, 41, 210),(11, 1, 133),(30, 96, 58),(230, 136, 109),(126, 45, 174),(164, 63, 165),(32, 111, 29),(232, 40, 70),(55, 31, 198),(148, 211, 129),(10, 186, 211),(181, 201, 94),(55, 35, 92),(129, 140, 233),(70, 250, 116),(61, 209, 152),(216, 21, 138),(100, 0, 176),(3, 42, 70),(151, 13, 44),(216, 102, 88),(125, 216, 93),(171, 236, 47),(253, 127, 103),(205, 137, 244),(193, 137, 224),(36, 152, 214),(17, 50, 238),(154, 165, 67),(114, 129, 60),(119, 24, 48),(73, 8, 110)]
  
  #plot each boxes
  for box in boxes:
    #add score in label if score=True
    if score :
      label = labels[int(box[-1])+1] + " " + str(round(100 * float(box[-2]),1)) + "%"
    else :
      label = labels[int(box[-1])+1]
    #filter every box under conf threshold if conf threshold setted
    if conf :
      if box[-2] > conf:
        color = colors[int(box[-1])]
        box_label(image, box, label, color)
    else:
      color = colors[int(box[-1])]
      box_label(image, box, label, color)

  #show image
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  cv2.imshow("win",image) 
  cv2.waitKey(0)
  cv2.destroyAllWindows()  
    

## test plate recognition

In [25]:
MAX_OUTPUT=1
for pred in preds:
    boxes=[]
    scores=[]
    img=pred.orig_img.copy()
    for box in pred.boxes:
        x1=box.xyxy[0][0].numpy().astype(np.int32)
        y1=box.xyxy[0][1].numpy().astype(np.int32)
        x2=box.xyxy[0][2].numpy().astype(np.int32)
        y2=box.xyxy[0][3].numpy().astype(np.int32)
        boxes.append([y1,x1,y2,x2])
        scores.append(box.conf[0].numpy())
        
    boxes=np.array(boxes)
    scores=np.array(scores)
    indices=tf.image.non_max_suppression(boxes,scores,MAX_OUTPUT)
    boxes=tf.gather(boxes,indices)
    for  box in boxes:
        y1,x1,y2,x2=box
        pt1=np.array([x1,y1]).astype(np.int32)
        pt2=np.array([x2,y2]).astype(np.int32)
        name=str(time.time())[-3:]
        path="J:/jupyter/plate_recognition/car_plate_fa_numbers/test/" +name+".jpg"
        cv2.imwrite(path,img[y1:y2,x1:x2])
        cv2.rectangle(img,pt1,pt2, (0,255,255), 1, cv2.LINE_AA) 
    cv2.imshow("win",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()